In [36]:
import numpy as np

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.18.0


In [37]:
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep='\t')

CougsInAzure	westus2	CougsInAzure


In [38]:
experiment_name = 'TestModel1'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [39]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target: cpu-cluster


In [40]:
import os
script_folder = os.path.join(os.getcwd(), "TestModel1")
os.makedirs(script_folder, exist_ok=True)

In [31]:
%%writefile $script_folder/train.py

import os
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import joblib
from azureml.core import Workspace, Dataset, Run
import azureml.core

subscription_id = '9d0dfa04-d2f8-4521-b945-b3a7dbf43946'
resource_group = 'CougsInAzure'
workspace_name = 'CougsInAzure'

ws = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(ws, name='Alpha_Dataset_Featurized')
df = dataset.to_pandas_dataframe()

X = df["Data"].tolist()
y = df["Label"].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2)

run = Run.get_context()

dt = DecisionTreeClassifier().fit(X_train, y_train)

clf = DecisionTreeClassifier().fit(X_train, y_train)

print('Predict the test set')
y_hat = clf.predict(X_test)

# calculate accuracy on the prediction
acc = np.average(y_hat == y_test)
print('Accuracy is', acc)

#run.log('regularization rate', np.float(args.reg))
run.log('accuracy', np.float(acc))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=clf, filename='outputs/TestModel1.pkl')

Overwriting /mnt/batch/tasks/shared/LS_root/mounts/clusters/andrewcompute/code/Users/capstonesipsml/Deployments/TestModel1/train.py


In [32]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
    
env = Environment("Test-env1")
cd = CondaDependencies.create(pip_packages=['azureml-defaults'], conda_packages = ['scikit-learn'])

env.python.conda_dependencies = cd

env.register(workspace = ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:20200821.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "Test-env1",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
  

In [33]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder,
                      script='train.py',
                      compute_target=compute_target,
                      environment=env)

In [34]:
run = exp.submit(config=src)
run

Experiment,Id,Type,Status,Details Page,Docs Page
TestModel1,TestModel1_1607130772_ecb9f8b0,azureml.scriptrun,Starting,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
model = run.register_model(model_name='TestModel1', model_path='outputs/TestModel1.pkl')
print(model.name, model.id, model.version, sep='\t')